# 공급량 데이터를 누적하여 저장하는 프로그램
+ Author: Stewart

## sql 데이터 추출

In [35]:
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

# 공급량 누적 테이블
query = 'SELECT date_id FROM supply_cumsum ORDER BY date_id DESC LIMIT 1'
cur.execute(query)

row = cur.fetchone()
sp_sum_last_date = row[0]     # 공급량누적 테이블에 저장된 데이터의 마지막 일자

# 공급량 테이블
query = 'SELECT date_id FROM supply WHERE date_id > %s ORDER BY date_id'
cur.execute(query, sp_sum_last_date)
rows = cur.fetchall()

beg_date = rows[0][0]
end_date = rows[-1][0]

conn.close()

In [38]:
print(f'누적 테이블에는 {sp_sum_last_date}까지 데이터가 저장되어 있으며, 이 날짜 이후에 공급량 테이블에는 {beg_date}부터 {end_date}까지 데이터가 있음')

누적 테이블에는 20230531까지 데이터가 저장되어 있으며, 이 날짜 이후에 공급량 테이블에는 20230601부터 20230630까지 데이터가 있음


아래 내용은 db내에 적재 되어있는 <b>date</b>와 <b>supply</b> 테이블을 활용해서 <b>SQL 쿼리문</b>으로 작성

In [46]:
import pandas as pd

engine = dbconnect.db_engine("DEMO_DM")
query = """
SELECT s.date_id AS date_id
		, supply_mj AS day_mj
		, SUM(sum(supply_mj)) OVER(PARTITION BY DATE_FORMAT(date_type, '%%y-%%m') ORDER BY DATE_FORMAT(date_type, '%%Y-%%m-%%d')) AS month_mj
		, SUM(SUM(supply_mj)) OVER(PARTITION BY quarter_6,DATE_FORMAT(date_type, '%%y') ORDER BY DATE_FORMAT(date_type, '%%Y-%%m-%%d')) AS quarter_6_mj
		, SUM(SUM(supply_mj)) OVER(PARTITION BY quarter_4,DATE_FORMAT(date_type, '%%y') ORDER BY DATE_FORMAT(date_type, '%%Y-%%m-%%d')) AS quarter_4_mj
		, SUM(SUM(supply_mj)) OVER(PARTITION BY DATE_FORMAT(date_type, '%%y') ORDER BY DATE_FORMAT(date_type, '%%Y-%%m-%%d')) AS  year_mj
		, supply_m3 AS day_m3
		, SUM(sum(supply_m3)) OVER(PARTITION BY DATE_FORMAT(date_type, '%%y-%%m') ORDER BY DATE_FORMAT(date_type, '%%Y-%%m-%%d')) AS month_m3
		, SUM(SUM(supply_m3)) OVER(PARTITION BY quarter_6,DATE_FORMAT(date_type, '%%y') ORDER BY DATE_FORMAT(date_type, '%%Y-%%m-%%d')) AS quarter_6_m3
		, SUM(SUM(supply_m3)) OVER(PARTITION BY quarter_4,DATE_FORMAT(date_type, '%%y') ORDER BY DATE_FORMAT(date_type, '%%Y-%%m-%%d')) AS quarter_4_m3
		, SUM(SUM(supply_m3)) OVER(PARTITION BY DATE_FORMAT(date_type, '%%y') ORDER BY DATE_FORMAT(date_type, '%%Y-%%m-%%d')) AS  year_m3
		FROM mdw.date AS d
			INNER JOIN mdw.supply AS s on s.date_id = d.date_id and s.date_id between %(date1)s and %(date2)s
	group BY DATE_FORMAT(date_type, '%%Y-%%m-%%d') ;
"""

# sql execution with parameters
df_data = pd.read_sql(sql=query, con=engine, params={'date1' : beg_date, 'date2' : end_date})
df_data #테이블 확인

,date_id,day_mj,month_mj,quarter_6_mj,quarter_4_mj,year_mj,day_m3,month_m3,quarter_6_m3,quarter_4_m3,year_m3
0,20230601,80271126,8.027113e+07,8.027113e+07,8.027113e+07,8.027113e+07,1887432,1887432.0,1887432.0,1887432.0,1887432.0
1,20230602,78088792,1.583599e+08,1.583599e+08,1.583599e+08,1.583599e+08,1837501,3724933.0,3724933.0,3724933.0,3724933.0
2,20230603,60904197,2.192641e+08,2.192641e+08,2.192641e+08,2.192641e+08,1434642,5159575.0,5159575.0,5159575.0,5159575.0
3,20230604,53719220,2.729833e+08,2.729833e+08,2.729833e+08,2.729833e+08,1264695,6424270.0,6424270.0,6424270.0,6424270.0
4,20230605,67448344,3.404317e+08,3.404317e+08,3.404317e+08,3.404317e+08,1584963,8009233.0,8009233.0,8009233.0,8009233.0
5,20230606,63783350,4.042150e+08,4.042150e+08,4.042150e+08,4.042150e+08,1499277,9508510.0,9508510.0,9508510.0,9508510.0
6,20230607,75074358,4.792894e+08,4.792894e+08,4.792894e+08,4.792894e+08,1765133,11273643.0,11273643.0,11273643.0,11273643.0
7,20230608,75545097,5.548345e+08,5.548345e+08,5.548345e+08,5.548345e+08,1777401,13051044.0,13051044.0,13051044.0,13051044.0
8,20230609,73343119,6.281776e+08,6.281776e+08,6.281776e+08,6.281776e+08,1722373,14773417.0,14773417.0,14773417.0,14773417.0
9,20230610,58065276,6.862429e+08,6.862429e+08,6.862429e+08,6.862429e+08,1364050,16137467.0,16137467.0,16137467.0,16137467.0


In [47]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date_id       30 non-null     object 
 1   day_mj        30 non-null     int64  
 2   month_mj      30 non-null     float64
 3   quarter_6_mj  30 non-null     float64
 4   quarter_4_mj  30 non-null     float64
 5   year_mj       30 non-null     float64
 6   day_m3        30 non-null     int64  
 7   month_m3      30 non-null     float64
 8   quarter_6_m3  30 non-null     float64
 9   quarter_4_m3  30 non-null     float64
 10  year_m3       30 non-null     float64
dtypes: float64(8), int64(2), object(1)
memory usage: 2.7+ KB


In [48]:
df_data.columns

Index(['date_id', 'day_mj', 'month_mj', 'quarter_6_mj', 'quarter_4_mj',
       'year_mj', 'day_m3', 'month_m3', 'quarter_6_m3', 'quarter_4_m3',
       'year_m3'],
      dtype='object')

## DB 내 테이블 적재

In [50]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_data.itertuples():
    sql = "insert into supply_cumsum(date_id, 일_mj, 월_mj, 6분기_mj, 4분기_mj, 연_mj, \
                                              일_m3, 월_m3, 6분기_m3, 4분기_m3, 연_m3) \
                              values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11]))

conn.commit()
conn.close()